To read before  
- PCA/SVD 
- Try Except in python  
- Latent semantic ANalysis  

In [4]:
#necessary imports 
import nltk 
import numpy as np  
import matplotlib.pyplot as plt  

from nltk.stem import WordNetLemmatizer 
from sklearn.decomposition import TruncatedSVD  

wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
#Loading Titles 
titles = [line.rstrip() for line in open('./data/latent_semantic_analysis/all_book_titles.txt')] 
stopwords = set(w.rstrip() for w in open('./data/stopwords.txt'))  
stopwords = stopwords.union({'introduction','edition','series','application'
                            'approach','card','access','package','plus','etext',
                            'brief','vol','fundamental','guide','essential','printed',
                            'third','second','fourth'}) #Adding more stop words specific to this dataset

In [5]:
'''Reading all Titles,Tokenizing it and appending it to dictionary'''
def my_tokenizer2(s): 
    s =s.lower()   #Lowercasing all the words
    tokens = nltk.tokenize.word_tokenize(s)  #Tokenize the test
    tokens = [t for t in tokens if len(t)>2] #only word len greater than 2 is useful 
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] 
    tokens = [t for t in tokens if t not in stopwords]
    tokens = [t for t in tokens if not any(c.isdigit() for c in t)]
    return tokens  

word_index_map = {} 
current_index = 0 
all_tokens = [] 
all_titles = [] 
index_word_map = [] 
for title in titles: 
    try: 
        title = title.encode('ascii','ignore')
        all_titles.append(title)
        tokens = my_tokenizer2(title) 
        all_tokens.append(tokens) #Basically all Book titles tokenized.
        for token in tokens:
            if token not in word_index_map: 
                word_index_map[token] = current_index
                current_index +=1
                index_word_map.append(token)
    except:
        pass 
 

In [9]:
#unsupervised learning (No label)
def tokens_to_vector(tokens):
    x = np.zeros(len(word_index_map)) 
    for t in tokens:
        i = word_index_map[t]
        x[i]=1  
    return x

N = len(all_tokens) #All book titles[columns]
D = len(word_index_map) #All possible words in all the titles[rows]
X = np.zeros((D,N))
i=0 
for tokens in all_tokens:
    X[:,i]=tokens_to_vector(tokens) #Here book titles are columns
    i +=1 

In [72]:
svd = TruncatedSVD()  
Z = svd.fit_transform(X)

In [ ]:
plt.scatter(Z[:,0],Z[:,1])
for i in xrange(D):
    plt.annotate(s=index_word_map[i],xy=(Z[i,0],Z[i,1]))

In [78]:
plt.axis([0,4,0,1])
plt.savefig('books_read.png')